In [1]:
import persistent_laplacians.persistent_laplacians as pl
from persistent_laplacians.cubical_complexes import get_filtration_data 
import numpy as np
np.random.seed(42)
image = np.random.rand(1, 1)

boundary_matrices, boundary_maps_index_dict, _ = get_filtration_data(image)

pl.smallest_eigenvalue(
    boundary_matrices,
    boundary_maps_index_dict,
    zero_tol=1e-6,
    use_scipy=True,
    use_stepwise_schur=False,
)

/var/folders/vw/75ns4gcj2pqgdh10pz8jrhjw0000gn/T/ipykernel_59788/1924482526.py:9: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  pl.smallest_eigenvalue(


{1: {(0, 0): []}}

In [2]:
from sklearn.datasets import fetch_openml
import numpy as np
x, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
num_samples = 50
x = x[:num_samples].reshape(-1, 28, 28).astype(np.float32) / 255.0
y = y[:num_samples].astype(np.uint8)

dataset = list(zip(x, y))

In [15]:
import persistent_laplacians.persistent_laplacians as pl
from persistent_laplacians.cubical_complexes import get_filtration_data 
import numpy as np

def get_nonpersistent_eigenvalues_from_image(image):
    boundary_matrices, boundary_maps_index_dict, subsampled_filtration_indices = get_filtration_data(image, num_indices=4)
    result = pl.smallest_eigenvalue(
        boundary_matrices,
        boundary_maps_index_dict,
        filtration_subsampling=subsampled_filtration_indices,
        zero_tol=1e-6,
        use_scipy=True,
        use_stepwise_schur=True,
    )
    print(result)
    # Filter result to nonpersistent dim 1 features
    nonpersistent_dim1 = [
        (k[0], v)
        for k, v in result[1].items()
        if k[0] == k[1]
    ]
    nonpersistent_dim1.sort(key=lambda x: x[0])
    # Return first element of each or zero if missing
    return np.array([vec[0] if vec else 0 for _, vec in nonpersistent_dim1])

In [16]:
get_nonpersistent_eigenvalues_from_image(x[0])  # Example for the first image

{1: {(28, 84): [0.016964610940348807], (28, 56): [0.016964610940348807], (0, 84): [0.01653685476973387], (0, 28): [0.01653685476973387], (0, 0): [0.01653685476973387], (56, 56): [0.01877398922800754], (28, 28): [0.016964610940348807], (84, 84): [0.011724085691280745], (0, 56): [0.01653685476973387], (56, 84): [0.01877398922800754]}}


array([0.01653685, 0.01696461, 0.01877399, 0.01172409])

In [11]:
from data_analysis.cross_validation import run_cross_validation

accuracies_nonpersistent = run_cross_validation(
    dataset=dataset,
    feature_extractor=get_nonpersistent_eigenvalues_from_image,
    n_splits=5,
    random_state=42
)

mean_acc = np.mean(accuracies_nonpersistent)
std_acc = np.std(accuracies_nonpersistent)
print(f"Cross-validated accuracies: {accuracies_nonpersistent}")
print(f"Mean accuracy: {mean_acc:.3f} ± {std_acc:.3f}")

Cross-validated accuracies: [0.1, 0.3, 0.1, 0.3, 0.2]
Mean accuracy: 0.200 ± 0.089
